In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import lxml

In [2]:
# IMPORT CSV of WATCHLIST - must save with UTF-8 encoding in Excel

watchlistDF = pd.read_csv('import/WATCHLIST.csv')
watchlistDF = watchlistDF.drop(['Position', 'Created','Modified','Description'], axis=1)
watchlistDF = watchlistDF[['Title','Year','Runtime (mins)','Directors','Genres','Const','URL','IMDb Rating','Num Votes','Your Rating', 'Date Rated']]
watchlistDF.head()

,Title,Year,Runtime (mins),Directors,Genres,Const,URL,IMDb Rating,Num Votes,Your Rating,Date Rated
0,Avengers: Endgame,2019,181,"Anthony Russo, Joe Russo","Action, Adventure, Sci-Fi",tt4154796,https://www.imdb.com/title/tt4154796/,8.6,551638,NaN,NaN
1,The Godfather,1972,175,Francis Ford Coppola,"Crime, Drama",tt0068646,https://www.imdb.com/title/tt0068646/,9.2,1464382,NaN,NaN
2,The Godfather: Part II,1974,202,Francis Ford Coppola,"Crime, Drama",tt0071562,https://www.imdb.com/title/tt0071562/,9.0,1018758,NaN,NaN
3,The Dark Knight,2008,152,Christopher Nolan,"Action, Crime, Drama, Thriller",tt0468569,https://www.imdb.com/title/tt0468569/,9.0,2097524,NaN,NaN
4,Pulp Fiction,1994,154,Quentin Tarantino,"Crime, Drama",tt0110912,https://www.imdb.com/title/tt0110912/,8.9,1673510,NaN,NaN


In [3]:
# ============= Search Result Scraping =============
url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,&num_votes=10000,&genres=comedy&lists=%21watchlist&count=250'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object
soup = BeautifulSoup(response.text)

# Get titles, urls, const from results
titleList = []
fullUrlList = []
constList = []

results = soup.find_all('h3', class_='lister-item-header')

for result in results:
    titleText = result.find('a').text
    titleList.append(titleText)
    
    #grab shortened URL
    aTag = result.find('a')
    url = aTag['href']
    
    #turn into full URL & add to list
    fullUrl = 'https://www.imdb.com' + url
    fullUrlList.append(fullUrl)

    #strip out const & add to list
    const = url.replace('/title/','')
    const = const.replace('/','')
    constList.append(const)


In [4]:
# Finding const
fullUrlList = []
constList = []

results = soup.find_all('h3', class_='lister-item-header')

for result in results:
    #grab shortened URL
    aTag = result.find('a')
    url = aTag['href']
    
    #turn into full URL & add to list
    fullUrl = 'https://www.imdb.com' + url
    fullUrlList.append(fullUrl)

    #strip out const & add to list
    const = url.replace('/title/','')
    const = const.replace('/','')
    constList.append(const)

constList

['tt7131622',
 'tt7798634',
 'tt8695030',
 'tt7343762',
 'tt2283336',
 'tt5113040',
 'tt1489887',
 'tt7040874',
 'tt6751668',
 'tt0448115',
 'tt2139881',
 'tt1979376',
 'tt1298644',
 'tt8079248',
 'tt4595882',
 'tt5884052',
 'tt5463162',
 'tt3501632',
 'tt2015381',
 'tt3104988',
 'tt6966692',
 'tt1618434',
 'tt0106677',
 'tt5028340',
 'tt0829482',
 'tt0804492',
 'tt1469304',
 'tt2802144',
 'tt1431045',
 'tt0112442',
 'tt4649466',
 'tt1119646',
 'tt6511932',
 'tt6513120',
 'tt2283362',
 'tt0147800',
 'tt1255919',
 'tt0163651',
 'tt5848272',
 'tt0172156',
 'tt1661275',
 'tt3896198',
 'tt7401588',
 'tt6266538',
 'tt1596363',
 'tt0099422',
 'tt0960144',
 'tt0118715',
 'tt0103639',
 'tt1981677',
 'tt3783958',
 'tt0117008',
 'tt2709768',
 'tt0377092',
 'tt1987680',
 'tt0780521',
 'tt2194499',
 'tt0114011',
 'tt6412452',
 'tt2005151',
 'tt6663582',
 'tt4463894',
 'tt6911608',
 'tt5164214',
 'tt0208092',
 'tt0364725',
 'tt3606756',
 'tt4971344',
 'tt0852713',
 'tt2294629',
 'tt7374948',
 'tt34

In [5]:
# Get years from results
yearList = []

results = soup.find_all('span', class_='lister-item-year text-muted unbold')

for result in results:
    year = result.text
    year = year[-5:]
    year = year.replace(')', '')
    yearList.append(year)

In [6]:
ratingList = []
runtimeList = []
genreList = []

# Get Rating
ratingResults = soup.find_all('span', class_='certificate')
for result in ratingResults:
    ratingList.append(result.text)

# Get Runtime
rumtimeResults = soup.find_all('span', class_='runtime')
for result in rumtimeResults:
    runtimeList.append(result.text)

# Get Genre
genreResults = soup.find_all('span', class_='genre')
for result in genreResults:
    genreList.append(result.text)

In [7]:
# Get Rating
ratingResults = soup.find_all('div', class_='inline-block ratings-metascore')

metaScoreList = []

for rating in ratingResults:
    metaRating = rating.find('span').text
    metaRating = metaRating.replace('        ','')
    metaScoreList.append(metaRating)

In [8]:
# Build Dataframe from Search
fullDF = pd.DataFrame(list(zip(titleList, yearList, runtimeList, ratingList, genreList, metaScoreList, constList, fullUrlList)),
                 columns = ['Title', 'Year', 'Runtime', 'Rating', 'Genres', 'MetaScore', 'Const', 'URL'])

# Format year and score as numbers
fullDF.loc[fullDF['Year'].notnull(), 'Year'] = fullDF.loc[fullDF['Year'].notnull(), 'Year'].apply(int)
fullDF.loc[fullDF['MetaScore'].notnull(), 'MetaScore'] = fullDF.loc[fullDF['MetaScore'].notnull(), 'MetaScore'].apply(int)

searchResultDF = fullDF
fullDF.head()

,Title,Year,Runtime,Rating,Genres,MetaScore,Const,URL
0,Once Upon a Time... in Hollywood,2019,161 min,R,"\nComedy, Drama",83,tt7131622,https://www.imdb.com/title/tt7131622/
1,Ready or Not,2019,95 min,R,"\nComedy, Horror, Mystery",63,tt7798634,https://www.imdb.com/title/tt7798634/
2,The Dead Don't Die,2019,104 min,R,"\nComedy, Fantasy, Horror",54,tt8695030,https://www.imdb.com/title/tt8695030/
3,Good Boys,2019,90 min,R,"\nAdventure, Comedy",60,tt7343762,https://www.imdb.com/title/tt7343762/
4,Men in Black: International,2019,114 min,PG-13,"\nAction, Adventure, Comedy",38,tt2283336,https://www.imdb.com/title/tt2283336/


In [9]:
watchlistDF.head()

,Title,Year,Runtime (mins),Directors,Genres,Const,URL,IMDb Rating,Num Votes,Your Rating,Date Rated
0,Avengers: Endgame,2019,181,"Anthony Russo, Joe Russo","Action, Adventure, Sci-Fi",tt4154796,https://www.imdb.com/title/tt4154796/,8.6,551638,NaN,NaN
1,The Godfather,1972,175,Francis Ford Coppola,"Crime, Drama",tt0068646,https://www.imdb.com/title/tt0068646/,9.2,1464382,NaN,NaN
2,The Godfather: Part II,1974,202,Francis Ford Coppola,"Crime, Drama",tt0071562,https://www.imdb.com/title/tt0071562/,9.0,1018758,NaN,NaN
3,The Dark Knight,2008,152,Christopher Nolan,"Action, Crime, Drama, Thriller",tt0468569,https://www.imdb.com/title/tt0468569/,9.0,2097524,NaN,NaN
4,Pulp Fiction,1994,154,Quentin Tarantino,"Crime, Drama",tt0110912,https://www.imdb.com/title/tt0110912/,8.9,1673510,NaN,NaN


In [10]:
# Compare watchlist & search results ---- x = search, y = watchlist
mergedDF = searchResultDF.merge(watchlistDF,on='Const', how='outer')

# Drop all non Nan rows (movies in the search, not in watchlist)
mergedDF = mergedDF[pd.isnull(mergedDF['Title_y'])]
mergedDF

# Create new DF for movies to watch
moviesToWatch = mergedDF
moviesToWatch = moviesToWatch[['Title_x', 'Year_x', 'Runtime', 'Rating', 'Genres_x', 'MetaScore', 'Const', 'URL_x']]
moviesToWatch = moviesToWatch.rename(columns={'Title_x':'Title', 'Year_x':'Year', 'Genres_x':'Genres', 'URL_x':'URL'})

# Remove \n in Genres
moviesToWatch = moviesToWatch.replace('\nComedy','Comedy')

moviesToWatch


,Title,Year,Runtime,Rating,Genres,MetaScore,Const,URL
0,Once Upon a Time... in Hollywood,2019,161 min,R,"\nComedy, Drama",83,tt7131622,https://www.imdb.com/title/tt7131622/
1,Ready or Not,2019,95 min,R,"\nComedy, Horror, Mystery",63,tt7798634,https://www.imdb.com/title/tt7798634/
2,The Dead Don't Die,2019,104 min,R,"\nComedy, Fantasy, Horror",54,tt8695030,https://www.imdb.com/title/tt8695030/
3,Good Boys,2019,90 min,R,"\nAdventure, Comedy",60,tt7343762,https://www.imdb.com/title/tt7343762/
4,Men in Black: International,2019,114 min,PG-13,"\nAction, Adventure, Comedy",38,tt2283336,https://www.imdb.com/title/tt2283336/
5,The Secret Life of Pets 2,2019,86 min,PG,"\nAnimation, Adventure, Comedy",55,tt5113040,https://www.imdb.com/title/tt5113040/
7,A Simple Favor,2018,117 min,R,"\nComedy, Crime, Drama",67,tt7040874,https://www.imdb.com/title/tt7040874/
8,Parasite,2019,132 min,R,"\nComedy, Drama, Thriller",91,tt6751668,https://www.imdb.com/title/tt6751668/
9,Shazam!,2019,132 min,PG-13,"\nAction, Adventure, Comedy",70,tt0448115,https://www.imdb.com/title/tt0448115/
10,Long Shot,2019,125 min,R,"\nComedy, Romance",67,tt2139881,https://www.imdb.com/title/tt2139881/


In [11]:
# Write Both DFs (Search & Watchlist) to XLSX
writer = pd.ExcelWriter('export/imdb_list_v2.xlsx', engine='xlsxwriter')

searchResultDF.to_excel(writer, sheet_name='Search Results', index=False)
watchlistDF.to_excel(writer, sheet_name='Watchlist', index=False)
moviesToWatch.to_excel(writer, sheet_name='Movies To Watch', index=False)

writer.save()